# CSVファイル出力マニュアル ( python )

## 1. データベースへの接続

はじめに，sqlite3の実行とcsv出力に必要なモジュールをインポートします．  
Pythonには標準でSQLiteがインストールされています．

In [ ]:
import sqlite3
import csv

今回，データベースの情報は「trial.sqlite3」というファイルに全て格納されているとします．  
( 実際は，配布されたデータベースファイルのファイル名に改めて実行してください．)

まず，以下のようにしてデータベースに接続します．

In [ ]:
# Google Colaboratoryで作業する場合に実行してください．  
# データの読み込み方が不明な場合は，別途配布している「GoogleColabにおけるドライブのマウントの仕方」と「Google Drive上でのフォルダ構造の説明」のpdfファイルをご覧ください．
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# データベースファイルのパスを渡してください
dbpath = '/content/drive/---/---/data.sqlite3'

In [ ]:
conn = sqlite3.connect(dbpath)

これで，データベースへの接続は完了です．  
※データベースが存在しない場合は，新規作成された上で接続されます．

## 2. データベースの観察

続いて，カーソルの取得をします．このモジュールでは，カーソルをイテレータとして扱います．  
カーソルを取得後にexecuteメソッドを用いると，SELECT文を実行することができます．

In [ ]:
cur = conn.cursor()

ここでは、データの出力に必要となるデータベースの情報，特にテーブル構造について確認します．  
データの抽出や加工については，SQL回の講義資料等を参照ください．

In [ ]:
# テーブル名の確認
cur.execute('select name from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

In [ ]:
# カラム名(CREATE TABLE文)の確認
cur.execute('select sql from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

In [ ]:
# 全情報の確認 (type, name, tbl_name, rootpage, sql)
cur.execute('select * from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

## 3. データの出力 (csv形式)

fetchoneメソッドで1件ずつ取得する場合

In [ ]:
cur.execute('select * from <テーブル名>')
print(cur.fetchone()) #1レコード目の取得
print(cur.fetchone()) #2レコード目の取得

fetchallメソッドで全件を取得する場合 ※実行時間が長いことがあります

In [ ]:
cur.execute('select * from <テーブル名>')
for row in cur.fetchall():
  print(row)

csv出力 ( [out] は任意のファイル名)

In [ ]:
#テーブルから特定の列を取得
cur.execute('select <カラム名> from <テーブル名>')
with open('out.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [ ]:
#テーブルから全ての列を取得
cur.execute('select * from <テーブル名>')
with open('out.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [ ]:
# 処理を確定
conn.commit()

左サイドバーのFilesに「out.csv」が出力されていることを確認してください．  
出力されたcsvファイルが確認出来たら，カーソルを閉じ，接続を閉じます．

In [ ]:
cur.close()

In [ ]:
conn.close()

## 【補足】出力の確認

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv('out.csv', sep=',')

以上で，ライブラリを用いたcsv出力は完了です．